In [2]:
import pickle
import copy
import numpy as np
import tempfile
import os

from simtk import unit, openmm
from openmmtools.constants import kB
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState

from perses.dispersed.utils import configure_platform
from perses.tests.utils import compute_potential_components
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT


conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2021.03.4 jupyter extensions


In [3]:
def test_bond_energies(htf, is_old=True):
    
    # Get original system and positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)
    
    # Get hybrid system, positions, and forces
    hybrid_system = htf.hybrid_system
    hybrid_positions = htf.hybrid_positions
#     force_dict = {force.__class__.__name__: force for force in hybrid_system.getForces()}
#     print(force_dict)
#     custom_bond_force = force_dict['CustomBondForce']
#     custom_angle_force = force_dict['CustomAngleForce']
#     custom_torsion_force = force_dict['CustomTorsionForce']
#     print("got forces")
    bond_force_index = 0
    angle_force_index = 1 
    torsion_force_index = 2 
    custom_bond_force = hybrid_system.getForce(bond_force_index)
    custom_angle_force = hybrid_system.getForce(angle_force_index)
    custom_torsion_force = hybrid_system.getForce(torsion_force_index)
#     custom_bond_force = htf._hybrid_system_forces['CustomBondForce']
#     custom_angle_force = htf._hybrid_system_forces['CustomAngleForce']
#     custom_torsion_force = htf._hybrid_system_forces['CustomTorsionForce']
    forces = [custom_bond_force, custom_angle_force, custom_torsion_force]
    force_names = ['bonds', 'angles', 'torsions']
    
    # Remove all other forces
    for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 4, -1):
        hybrid_system.removeForce(i - 1)
    
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    for force, name in zip(forces, force_names):
        for i in range(force.getNumGlobalParameters()):
            if force.getGlobalParameterName(i) == f'lambda_alchemical_{name}_old':
                force.setGlobalParameterDefaultValue(i, lambda_old)
            if force.getGlobalParameterName(i) == f'lambda_alchemical_{name}_new':
                force.setGlobalParameterDefaultValue(i, lambda_new)

    # Zero the unique old/new terms at lambda = 1/0
    hybrid_to_bond_indices = htf._hybrid_to_new_bond_indices if is_old else htf._hybrid_to_old_bond_indices
    hybrid_to_angle_indices = htf._hybrid_to_new_angle_indices if is_old else htf._hybrid_to_old_angle_indices
    hybrid_to_torsion_indices = htf._hybrid_to_new_torsion_indices if is_old else htf._hybrid_to_old_torsion_indices
    for hybrid_idx, idx in hybrid_to_bond_indices.items():
        p1, p2, hybrid_params = custom_bond_force.getBondParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
        hybrid_params[-2] *= 0 # zero K_old
        hybrid_params[-1] *= 0 # zero K_new
        custom_bond_force.setBondParameters(hybrid_idx, p1, p2, hybrid_params)
    for hybrid_idx, idx in hybrid_to_angle_indices.items():
        p1, p2, p3, hybrid_params = custom_angle_force.getAngleParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
        hybrid_params[-2] *= 0
        hybrid_params[-1] *= 0
        custom_angle_force.setAngleParameters(hybrid_idx, p1, p2, p3, hybrid_params)
    for hybrid_idx, idx in hybrid_to_torsion_indices.items():
        p1, p2, p3, p4, hybrid_params = custom_torsion_force.getTorsionParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
        hybrid_params[-2] *= 0
        hybrid_params[-1] *= 0
        custom_torsion_force.setTorsionParameters(hybrid_idx, p1, p2, p3, p4, hybrid_params)

    # Get energy components of original system
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    # Get energy components of hybrid system
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)
    
    assert np.isclose([components_other[0][1]], [components_hybrid[0][1]])
    
    print("Success! Custom bond force and standard bond force energies are equal!")
    


In [4]:
def test_RESTCapableHybridTopologyFactory_energies():
    # Create a htf
    atp, system_generator = generate_atp(phase = 'vacuum')
    htf = generate_dipeptide_top_pos_sys(atp.topology, 
                                   'THR', 
                                   atp.system, 
                                   atp.positions, 
                                   system_generator,
                                   conduct_htf_prop=True,
                                   generate_rest_capable_hybrid_topology_factory=True,
                                   validate_endstate_energy=False)

#     with tempfile.TemporaryDirectory() as temp_dir:
#         os.chdir(temp_dir)
    print ('finished htf')

    with open("atp_vacuum.pickle", "wb") as f:
        pickle.dump(htf, f)
    
    print ('saved htf')
    
    # Test at lambda = 0
    with open("atp_vacuum.pickle", "rb") as f:
        htf = pickle.load(f)
        
    print('loaded htf')

    test_bond_energies(htf)

    # Test at lambda = 1
    with open("atp_vacuum.pickle", "rb") as f:
        htf = pickle.load(f)

    test_bond_energies(htf, is_old=False)

In [5]:
test_RESTCapableHybridTopologyFactory_energies()

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 14, 12: 15}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 7
INFO:geometry:Atom index proposal order is [18, 14, 19, 13, 15, 17, 16]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions
INFO:geometry:There are 7 new atoms
INFO:geometry:	reduced angle potential = 0.6176356126052863.


conducting subsequent work with the following platform: CUDA


INFO:geometry:	reduced angle potential = 0.0448283056189217.
INFO:geometry:	reduced angle potential = 0.008198077456058135.
INFO:geometry:	reduced angle potential = 0.268900888526781.
INFO:geometry:	reduced angle potential = 0.45387795337521697.
INFO:geometry:	reduced angle potential = 0.04102923556287967.
INFO:geometry:	reduced angle potential = 1.4958462454327994.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined wi

...

INFO:geometry:total reduced potential before atom placement: 16.81406823661637


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.00017810081275281765), ('CustomTorsionForce', 0.004336815681512571), ('CustomBondForce', 8.812010530371548)]


INFO:geometry:total reduced energy added from growth system: 8.816525446865816
INFO:geometry:final reduced energy 25.630593424131863
INFO:geometry:sum of energies: 25.63059368348219
INFO:geometry:magnitude of difference in the energies: 2.5935032432755634e-07
INFO:geometry:Final logp_proposal: -27109.780649617715
INFO:relative:*** Generating RESTCapableHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:r_cutoff is 100 nm
INFO:relative:alpha_ewald is 0
INFO:relative:w_scale is 0.1
INFO:relative:Creating hybrid system
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPer

finished htf
saved htf
loaded htf
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0345517625414457), ('HarmonicAngleForce', 0.6072023625714801), ('PeriodicTorsionForce', 16.176828968049836), ('NonbondedForce', -39.186928227296285), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.3682025990203521), ('CustomAngleForce', 5.406361776360386), ('CustomTorsionForce', 24.89053794480953), ('AndersenThermostat', 0.0)]


AssertionError: 